# Sample Script to update testplan using a template

## Description 
- This script automates the process of updating test plans by applying a predefined template.
- It takes a list of test plan IDs and a template ID to override the selected test plan properties with the values specified in the template.  

## Limitations
- Only fields supported for updates by the test plans backend API will be modified. 
  
Note: To customize the automate algorithm, modify the following sections:   
- [APIs and Algorithms](#APIs-and-algorithms)
- [Actions and Output](#Actions-and-output) 

## Imports

In [ ]:
import requests
import os
import scrapbook as sb

## Parameters

- **`test_plan_ids`**: IDs of the test plans to be updated.
- **`template_id`**: ID of the test plan template used to update the test plans.

Parameters are also listed in the metadata for the **parameters cell**, along with their default values.  
The Notebook Execution services use that metadata to pass parameters to this notebook.  
To view the metadata:
- Select the code cell
- Click the **wrench icon** in the right panel.

### Sample metadata

```json
{
  "papermill": {
    "parameters": {
        "template_id": "",
        "test_plan_ids": []
    }
  },
  "systemlink": {
    "parameters": [
        {
            "display_name": "Test plan IDs",
            "id": "test_plan_ids",
            "type": "string[]"
        },
        {
            "display_name": "Test plan template ID",
            "id": "template_id",
            "type": "string"
        }
    ],
    "version": 1
  },
  "tags": ["parameters"]
}
```
For more information on how parameterization works, review the [papermill documentation](https://papermill.readthedocs.io/en/latest/usage-parameterize.html#how-parameters-work).

In [ ]:
test_plan_ids = ["1510059", "1510157"] #Add the test plans ids here
template_id = "" #Add the test plan template id here

## Constants

In [ ]:
x_ni_api_key = os.getenv("SYSTEMLINK_API_KEY")
sl_uri = os.getenv("SYSTEMLINK_HTTP_URI")

headers = {
    'Content-Type': 'application/json',
    'x-ni-api-key': x_ni_api_key
}

## Models

In [ ]:
# Base class for TestPlan and TestPlanTemplate

class TestPlanBase:
    def __init__(self, id, description, test_program, estimated_duration_in_seconds, 
                 system_filter=None, execution_actions=None, properties=None, dashboard=None):
        self.id = id
        self.description = description
        self.test_program = test_program
        self.estimated_duration_in_seconds = estimated_duration_in_seconds
        self.system_filter = system_filter
        self.execution_actions = execution_actions or []
        self.properties = properties or {}
        self.dashboard = dashboard or {}

    def __repr__(self):
        return f'{self.__class__.__name__}(id={self.id}, description={self.description}, test_program={self.test_program}, estimated_duration_in_seconds={self.estimated_duration_in_seconds}, system_filter={self.system_filter}, execution_actions={self.execution_actions}, properties={self.properties}, dashboard={self.dashboard})'

In [ ]:
class TestPlan(TestPlanBase):
    def to_dict(self):
        """Convert the TestPlan object to a dictionary for JSON serialization."""
        return {
            "id": self.id,
            "description": self.description,
            "testProgram": self.test_program,
            "estimatedDurationInSeconds": self.estimated_duration_in_seconds,
            "systemFilter": self.system_filter,
            "executionActions": self.execution_actions,
            "properties": self.properties,
            "dashboard": self.dashboard
        }

In [ ]:
class TestPlanTemplate(TestPlanBase):
    pass

## APIs and algorithms

In [ ]:
def query_test_plans(test_plan_ids):
    query_test_plans_url = f'{sl_uri}/niworkorder/v1/query-testplans'
    filter_conditions = ' || '.join([f'id == "{id}"' for id in test_plan_ids])
    request_payload = {"filter": filter_conditions}

    print(f"Fetching test plans: {', '.join(test_plan_ids)}")
    response = requests.post(query_test_plans_url, headers=headers, json=request_payload)

    if response.status_code != 200:
        print(f"Error fetching test plans {', '.join(test_plan_ids)}: {response.status_code} - {response.text}")
        return None

    response_data = response.json()
    test_plans_data = response_data.get('testPlans', [])

    return [TestPlan(
        id=test_plan_data.get('id'),
        description=test_plan_data.get('description', ''),
        test_program=test_plan_data.get('testProgram', {}),
        estimated_duration_in_seconds=test_plan_data.get('estimatedDurationInSeconds', 0),
        system_filter=test_plan_data.get('systemFilter', None),
        execution_actions=test_plan_data.get('executionActions', []),
        properties=test_plan_data.get('properties', {}),
        dashboard=test_plan_data.get('dashboard', {})
    ) for test_plan_data in test_plans_data]

In [ ]:
def fetch_test_plan_template(template_id):
    template_url = f'{sl_uri}/niworkorder/v1/query-testplan-templates'
    request_payload = { "filter": f'id == \"{template_id}\"' }

    print(f"Fetching test plan template: {template_id}")
    response = requests.post(template_url, headers=headers, json=request_payload)

    if response.status_code != 200:
        print(f"Error fetching test plan template {template_id}: {response.status_code} - {response.text}")
        return None

    response_data = response.json()
    templates = response_data.get('testPlanTemplates', [])
    
    if not templates:
        print(f"No template found with ID {template_id}")
        return None
        
    template_data = templates[0]
    return TestPlanTemplate(
        id=template_data.get('id'),
        description=template_data.get('description', ''),
        test_program=template_data.get('testProgram', {}),
        estimated_duration_in_seconds=template_data.get('estimatedDurationInSeconds', 0),
        system_filter=template_data.get('systemFilter', None),
        execution_actions=template_data.get('executionActions', []),
        properties=template_data.get('properties', {}),
        dashboard=template_data.get('dashboard', {})
    )

In [ ]:
def update_test_plans(test_plans):
    update_test_plans_url = f'{sl_uri}/niworkorder/v1/update-testplans'
    test_plans_dicts = [test_plan.to_dict() for test_plan in test_plans]
    request_payload = {"testPlans": test_plans_dicts, "replace": True }

    response = requests.post(update_test_plans_url, headers=headers, json=request_payload)
    test_plans_data = response.json()

    if response.status_code != 200:
        print(f"Error updating test plans: {response.status_code} - {response.text}")

    if 'updatedTestPlans' in test_plans_data:
        updated_test_plan_ids = [test_plan['id'] for test_plan in test_plans_data['updatedTestPlans']]

    return updated_test_plan_ids

In [ ]:
def update_test_plans_with_template(test_plans, test_plan_template):
    if not test_plans or not test_plan_template:
        return []

    for test_plan in test_plans:
        test_plan.description = test_plan_template.description
        test_plan.test_program = test_plan_template.test_program
        test_plan.estimated_duration_in_seconds = test_plan_template.estimated_duration_in_seconds
        test_plan.system_filter = test_plan_template.system_filter
        test_plan.execution_actions = test_plan_template.execution_actions
        test_plan.properties = test_plan_template.properties
        test_plan.dashboard = test_plan_template.dashboard
    
    updated_test_plan_ids = update_test_plans(test_plans)
    return updated_test_plan_ids

## Actions and output

In [ ]:
updated_test_plan_ids = []
failed_test_plan_ids = []

if not template_id:
    print("No template ID provided")
    sb.glue("Failed to fetch template", "No template ID provided")
else:    
    # Fetch test plan template
    test_plan_template = fetch_test_plan_template(template_id)
    
    if not test_plan_template:
        sb.glue("Failed to fetch template", template_id)
    else:
        # Fetch test plans
        test_plans = query_test_plans(test_plan_ids)
        
        # Update test plans with template
        updated_test_plan_ids = update_test_plans_with_template(test_plans, test_plan_template)
        failed_test_plan_ids = list(set(test_plan_ids) - set(updated_test_plan_ids))

        # Output
        print("Total test plans:", len(test_plan_ids))
        print("Test plans updated:", ', '.join(updated_test_plan_ids) if updated_test_plan_ids else "--")
        print("Test plans not updated:", ', '.join(failed_test_plan_ids) if failed_test_plan_ids else "--")
        
        # Executions page output
        sb.glue("Total test plans:", len(test_plan_ids))
        sb.glue("Test plans updated:", ', '.join(updated_test_plan_ids) if updated_test_plan_ids else "--")
        sb.glue("Test plans not updated:", ', '.join(failed_test_plan_ids) if failed_test_plan_ids else "--")

## Next Steps
- Publish this notebook to SystemLink by right-clicking it in the JupyterLab File Browser with the interface as Test Plan Automations.